In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import logging
import sys
import re
from citymobil_python_mysql_wrapper import MysqlWrapper
import pyexasol
from random import randrange
from itertools import combinations

import statsmodels.api as sm
from statsmodels.stats.power import tt_ind_solve_power, GofChisquarePower
from statsmodels.stats.gof import chisquare_effectsize
from statsmodels.sandbox.stats.multicomp import multipletests
from scipy.stats import norm, chi2_contingency

In [2]:
import query as q

In [3]:
pd.set_option("display.max_columns", 999)
pd.set_option("display.max_rows", 999)

In [4]:
def get_bootstrap(
    data_column_1, # числовые значения первой выборки
    data_column_2, # числовые значения второй выборки
    boot_it = 1000, # количество бутстрэп-подвыборок
    statistic = np.mean, # интересующая нас статистика
    bootstrap_conf_level = 0.99 # уровень значимости
):
    
    '''Bootstrap для непрерывной метрики'''
    
    boot_data = []
    for i in range(boot_it): # извлекаем подвыборки
#    for i in tqdm(range(boot_it)): # извлекаем подвыборки
        samples_1 = data_column_1.sample(
            len(data_column_1), 
            replace = True # параметр возвращения
        ).values
        
        samples_2 = data_column_2.sample(
            len(data_column_1), 
            replace = True
        ).values
        
        boot_data.append(statistic(samples_1)-statistic(samples_2)) # mean() - применяем статистику
        
    pd_boot_data = pd.DataFrame(boot_data)
        
    left_quant = (1 - bootstrap_conf_level)/2
    right_quant = 1 - (1 - bootstrap_conf_level) / 2
    quants = pd_boot_data.quantile([left_quant, right_quant])
        
    p_1 = norm.cdf(
        x = 0, 
        loc = np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_2 = norm.cdf(
        x = 0, 
        loc = -np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_value = min(p_1, p_2) * 2
       
    return {"boot_data": boot_data, 
            "quants": quants, 
            "p_value": p_value}

def bootstrap_ratio(
        data: pd.DataFrame,
        x: str,
        y: str,
        x_f,
        y_f,
        split='EXP_GROUP',
        user_level_col='DRIVER_RK',
        boot_it=1000,
        conf_level=0.99):
    
    '''Bootstrap для метрики отношения'''
    
    data = data.sort_values(by=[split], ascending=[True])
    
    data_splitted = [x for _, x in data.groupby(split)]
    boot_data = []

    for i in range(boot_it):
        s0 = data_splitted[0][data_splitted[0][user_level_col].isin(
            data_splitted[0][user_level_col].sample(data_splitted[0][user_level_col].nunique(), replace=True))]
        s1 = data_splitted[1][data_splitted[1][user_level_col].isin(
            data_splitted[1][user_level_col].sample(data_splitted[1][user_level_col].nunique(), replace=True))]

        y0 = y_f(s0[y])
        y1 = y_f(s1[y])
        x0 = x_f(s0[x])
        x1 = x_f(s1[x])

        if y0 == 0 or y1 == 0:
            return None, False
        elif x0 == 0 or x1 == 0:
            return None, False

        s0_ratio = x0 / y0
        s1_ratio = x1 / y1
        boot_data.append(s1_ratio - s0_ratio)

    pd_boot_data = pd.DataFrame(boot_data)

    p_1 = norm.cdf(x=0, loc=np.mean(boot_data), scale=np.std(boot_data))
    p_2 = norm.cdf(x=0, loc=-np.mean(boot_data), scale=np.std(boot_data))
    pvalue = min(p_1, p_2) * 2
    mark = (pvalue < 1 - conf_level)

    return pvalue, mark

# Запрашиваем из БД все необходимые данные

In [5]:
cred = pd.read_json(r'/Users/skostuchik/crd_exa.json')
user = cred.iloc[0, 0]
password = cred.iloc[0, 1]

C = pyexasol.connect(dsn='ex1..3.city-srv.ru:8563', user=user, password=password, fetch_dict=True)

In [6]:
alpha, power, n_bins = 0.01, 0.9, 2

In [7]:
campaigns = [
    {'id': 50535, 'date_from' : '2022-02-16', 'date_to' : '2022-02-22'},
    {'id': 50735, 'date_from' : '2022-02-18', 'date_to' : '2022-02-24'}
    ]

In [8]:
ar_suggest_data, of2r_data, mph_data, copt_data, dist_data, churn_data = [pd.DataFrame() for i in range(6)]

In [9]:
for c in tqdm(campaigns):
    query_params = {'id':c.get('id'), 'date_from':c.get('date_from'), 'date_to':c.get('date_to')}
    
    e_ar, e_of2r, e_mph, e_copt, e_dist, e_ch = C.execute(q.ar_query.format(**query_params)),\
    C.execute(q.of2r_query.format(**query_params)), C.execute(q.mph_query.format(**query_params)),\
    C.execute(q.copt_query.format(**query_params)), C.execute(q.dist_query.format(**query_params)),\
    C.execute(q.churn_query.format(**query_params))
    
    ar_suggest_data, of2r_data, mph_data, copt_data, dist_data, churn_data = \
    ar_suggest_data.append(pd.DataFrame(e_ar.fetchall())), of2r_data.append(pd.DataFrame(e_of2r.fetchall())),\
    mph_data.append(pd.DataFrame(e_mph.fetchall())), copt_data.append(pd.DataFrame(e_copt.fetchall())),\
    dist_data.append(pd.DataFrame(e_dist.fetchall())), churn_data.append(pd.DataFrame(e_ch.fetchall()))

100%|██████████| 2/2 [00:03<00:00,  1.98s/it]


In [10]:
for dataframe in [ar_suggest_data, of2r_data, mph_data, copt_data, dist_data, churn_data]:
    dataframe.reset_index(inplace=True)
    dataframe.drop(columns='index', inplace=True)

In [11]:
titles_sql = f'''select id, TITLE
from REPLICA.DRIVER_BONUS db
    where id in {tuple([c.get('id') for c in campaigns])}'''

titles = pd.DataFrame(C.execute(titles_sql).fetchall())

titles_dict = {}
for i in titles.itertuples():
    titles_dict.update({i.ID:i.TITLE})

In [12]:
split_pairs_df = ar_suggest_data.groupby('BONUS_ID')['EXP_GROUP'].unique().reset_index()
split_pairs = {}

for r in split_pairs_df.itertuples():
    split_pairs.update(
        {r.BONUS_ID:{comb[0] + '_' + comb[1]:list(comb) for comb in combinations(sorted(r.EXP_GROUP), 2)}})

In [13]:
#есть косяк с наполнением таблицы SH в DWH
mph_data = mph_data[mph_data['SUPPLY_HOURS'].notna()]

In [14]:
config = {
    'metrics': {
        'AR':{
            'df':ar_suggest_data,
            'add_breakdown':{
                'Total':'Total',
                'ChainOffer':'SPECIFICATION_NAME',
                '0 - 3 km':'DISTANCE_CATEGORY','3 - 6 km':'DISTANCE_CATEGORY',
                '6 - 10 km':'DISTANCE_CATEGORY','+10 km':'DISTANCE_CATEGORY'
            },
            'options':{
                'x':['ACCEPT'],
                'n':['FSS_ID', pd.Series.nunique]
            },
            'type':'binomial'
        },
        'RR':{
            'df':ar_suggest_data,
            'add_breakdown':{
                'Total':'Total',
                'ChainOffer':'SPECIFICATION_NAME',
                '0 - 3 km':'DISTANCE_CATEGORY','3 - 6 km':'DISTANCE_CATEGORY',
                '6 - 10 km':'DISTANCE_CATEGORY','+10 km':'DISTANCE_CATEGORY'
            },
            'options':{
                'x':['REJECT'],
                'n':['FSS_ID', pd.Series.nunique]
            },
            'type':'binomial'
        },
        'FR':{
            'df':ar_suggest_data,
            'add_breakdown':{
                'Total':'Total',
                'ChainOffer':'SPECIFICATION_NAME',
                '0 - 3 km':'DISTANCE_CATEGORY','3 - 6 km':'DISTANCE_CATEGORY',
                '6 - 10 km':'DISTANCE_CATEGORY','+10 km':'DISTANCE_CATEGORY'
            },
            'options':{
                'x':['FRAUD'],
                'n':['FSS_ID', pd.Series.nunique]
            },
            'type':'binomial'
        },
        'CHURN_7DAY':{
            'df':churn_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['CHURN_7DAY'],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'binomial'
        },
        'CHURN_14DAY':{
            'df':churn_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['CHURN_14DAY'],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'binomial'
        },
        'CHURN_21DAY':{
            'df':churn_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['CHURN_21DAY'],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'binomial'
        },
        'MPH_GROSS':{
            'df':mph_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['MONEY_GROSS', np.sum],
                'y':['SUPPLY_HOURS', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'MPH_NET':{
            'df':mph_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['MONEY_NET', np.sum],
                'y':['SUPPLY_HOURS', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'ORGANIC_MPH_GROSS':{
            'df':mph_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['ORGANIC_MONEY_GROSS', np.sum],
                'y':['SUPPLY_HOURS', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'ORGANIC_MPH_NET':{
            'df':mph_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['ORGANIC_MONEY_NET', np.sum],
                'y':['SUPPLY_HOURS', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'SHpD':{
            'df':mph_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['SUPPLY_HOURS', np.sum],
                'y':['DRIVER_RK', pd.Series.nunique],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'DE':{
            'df':mph_data,
            'add_breakdown':{
                'Total':'Total', 'Not_park':'PARK', 'Park':'PARK', 'not_brand':'BRAND', 'brand':'BRAND',
                'TOP':'SEGMENT', 'HIGH':'SEGMENT'
            },
            'options':{
                'x':['ON_TRIP', np.sum],
                'y':['SUPPLY_HOURS', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'TPD':{
            'df':mph_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['TRIPS', np.sum],
                'y':['DRIVER_RK', pd.Series.nunique],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'OF2R':{
            'df':of2r_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['RIDES', np.sum],
                'y':['OFFERS', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'GMVPT':{
            'df':copt_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['GMV', np.sum],
                'y':['RIDES', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'COPT':{
            'df':copt_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['CONTRIBUTION', np.sum],
                'y':['RIDES', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'DIPT':{
            'df':copt_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['DI', np.sum],
                'y':['RIDES', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'COMPT':{
            'df':copt_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['COMMISSION', np.sum],
                'y':['RIDES', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'DIST_MEAN':{
            'df':dist_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['EXP_DIST_KM', np.mean],
                'n':['ORDER_RK', pd.Series.nunique]
            },
            'type':'continious'
        },
        'DIST_MEDIAN':{
            'df':dist_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['EXP_DIST_KM', np.median],
                'n':['ORDER_RK', pd.Series.nunique]
            },
            'type':'continious'
        }
    }
}

In [15]:
def stat_res_calculation(metric: str, m_type: str, df: pd.DataFrame, splits: list, cfg=config):
    '''Возвращает массив со всеми расчитанными метриками'''
    
    split_1, split_2 = splits
    
    if m_type == 'binomial':
        a_s = len(df[(df['EXP_GROUP'] == split_1)&(df[config['metrics'][metric]['options']['x'][0]] > 0)])
        a_f = len(df[(df['EXP_GROUP'] == split_1)&(df[config['metrics'][metric]['options']['x'][0]] == 0)])
        b_s = len(df[(df['EXP_GROUP'] == split_2)&(df[config['metrics'][metric]['options']['x'][0]] > 0)])
        b_f = len(df[(df['EXP_GROUP'] == split_2)&(df[config['metrics'][metric]['options']['x'][0]] == 0)])
        
        act_nobs_a, act_nobs_b = a_s+a_f, b_s+b_f
        successes, fails = [a_s, b_s], [a_f, b_f]
        probs0, probs1 = np.array([a_s, a_f]), np.array([b_s, b_f])
        effect_size = chisquare_effectsize(probs0, probs1, correction=None, cohen=True, axis=0)
        
        p_val = chi2_contingency(np.array([fails, successes]))[1]
        mde = int(GofChisquarePower().solve_power(effect_size=effect_size, alpha=alpha,
                                                  power=power, n_bins=n_bins))
        mde_marker_value = 1 if (act_nobs_a>=mde)&(act_nobs_b>=mde) else np.nan
        
        a = a_s/act_nobs_a
        b = b_s/act_nobs_b
        f = config['metrics'][metric]['options']['n'][1]
        a_drivers = df[df['EXP_GROUP'] == split_1]['DRIVER_RK'].nunique()
        b_drivers = df[df['EXP_GROUP'] == split_2]['DRIVER_RK'].nunique()
        lift = b-a

        
    elif m_type == 'ratio':
        x = config['metrics'][metric]['options']['x'][0]
        x_f = config['metrics'][metric]['options']['x'][1]
        y = config['metrics'][metric]['options']['y'][0]
        y_f = config['metrics'][metric]['options']['y'][1]
        
        f = config['metrics'][metric]['options']['n'][1]
        act_nobs_a = f(df[df['EXP_GROUP'] == split_1][config['metrics'][metric]['options']['n'][0]])
        act_nobs_b = f(df[df['EXP_GROUP'] == split_2][config['metrics'][metric]['options']['n'][0]])
        
        mde_ttl = df.groupby('EXP_GROUP').mean().reset_index()
        mean = df[df['EXP_GROUP']==split_1][x].mean()
        sd = df[df['EXP_GROUP']==split_1][x].std()
        lift = mde_ttl[x][1] / mde_ttl[x][0]-1
        effect_size = mean / sd * lift
        
        p_val = bootstrap_ratio(data=df, x=x, y=y, x_f=x_f, y_f=y_f)[0]
        mde = int(tt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=power, nobs1=None, ratio=1))
        mde_marker_value = 1 if (act_nobs_a>=mde)&(act_nobs_b>=mde) else np.nan
        
        a = x_f(df[df['EXP_GROUP']==split_1][x]) / y_f(df[df['EXP_GROUP']==split_1][y])
        b = x_f(df[df['EXP_GROUP']==split_2][x]) / y_f(df[df['EXP_GROUP']==split_2][y])
        a_drivers, b_drivers = act_nobs_a, act_nobs_b
        lift = (b-a)/abs(a)
        
    elif m_type == 'continious':
        x = config['metrics'][metric]['options']['x'][0]
        f = config['metrics'][metric]['options']['x'][1]
        f_n = config['metrics'][metric]['options']['n'][1]

        a = f(df[df['EXP_GROUP']==split_1][x])
        b = f(df[df['EXP_GROUP']==split_2][x])



        act_nobs_a = f_n(df[df['EXP_GROUP'] == split_1][config['metrics'][metric]['options']['n'][0]])
        act_nobs_b = f_n(df[df['EXP_GROUP'] == split_2][config['metrics'][metric]['options']['n'][0]])

        #mde_ttl = df.groupby('EXP_GROUP').mean().reset_index()
        mean = a
        sd = df[df['EXP_GROUP']==split_1][x].std()
        lift = b / a - 1
        effect_size = mean / sd * lift

        booted_data_ab = get_bootstrap(data_column_1=df[df['EXP_GROUP']==split_1][x], # числовые значения первой выборки
                                       data_column_2=df[df['EXP_GROUP']==split_2][x], # числовые значения второй выборки
                                       statistic = f)
        p_val = booted_data_ab["p_value"]
        q_low = float(booted_data_ab["quants"].iloc[0])
        q_high = float(booted_data_ab["quants"].iloc[1])

        if q_low < 0 and q_high > 0:
            p_val = 1
        else:
            p_val = p_val

        try:
            mde = int(tt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=power, nobs1=None, ratio=1))
        except:
            mde = 990999999
        mde_marker_value = 1 if (act_nobs_a>=mde)&(act_nobs_b>=mde) else np.nan

        a_drivers = df[df['EXP_GROUP'] == split_1]['DRIVER_RK'].nunique()
        b_drivers = df[df['EXP_GROUP'] == split_2]['DRIVER_RK'].nunique()

    else:
        return('Unknown metric type')
    
    nobs_diff_value = (act_nobs_b-act_nobs_a)/abs(act_nobs_a)
    dn_diff_value = (b_drivers-a_drivers)/abs(a_drivers)
    
    return({'p_val':p_val, 'nobs_needed':mde, 'mde_ok':mde_marker_value,
            'split_a':a, 'split_b':b, 'lift':lift, 'split_a_n_obs':act_nobs_a, 'split_b_n_obs':act_nobs_b,
            'n_obs_difference':nobs_diff_value, 'split_a_drivers':a_drivers, 'split_b_drivers':b_drivers,
            'unique_drivers_difference':dn_diff_value})

In [31]:
result_dict = {}

for b in tqdm(ar_suggest_data.BONUS_ID.unique()):
    for metric,v in config.get('metrics').items():
        for split_pair, splits in split_pairs.get(b).items():
            df = v.get('df')
            metric_data = df[(df['BONUS_ID']==int(b))&(df.EXP_GROUP.isin(splits))]
            for value, fltr in v.get('add_breakdown').items():
                if fltr != 'Total':
                    add_breakdown_data = metric_data[metric_data[fltr] == value]
                else:
                    add_breakdown_data = metric_data
                    
                if len(add_breakdown_data) == 0:
                    continue
                    
                BONUS_ID = add_breakdown_data['BONUS_ID'].iloc[0]

                metric_name = metric+'_'+value if len(v.get('add_breakdown')) > 1 else metric

                try:
                    src = stat_res_calculation(metric=metric, m_type=v.get('type'), df=add_breakdown_data,
                                               splits = splits)
                    result_dict.update({(BONUS_ID,split_pair,metric_name):src})
                except:
                    continue

  0%|          | 0/2 [00:00<?, ?it/s]<ipython-input-15-665ab6021a38>:43: RuntimeWarning: invalid value encountered in double_scalars
  lift = mde_ttl[x][1] / mde_ttl[x][0]-1
/Users/skostuchik/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1919: RuntimeWarning: invalid value encountered in double_scalars
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/skostuchik/opt/anaconda3/lib/python3.8/site-packages/statsmodels/stats/power.py:339: HypothesisTestWarning: Warning: Effect size of 0 detected
  warnings.warn('Warning: Effect size of 0 detected', HypothesisTestWarning)
 50%|█████     | 1/2 [00:27<00:27, 27.23s/it]<ipython-input-15-665ab6021a38>:43: RuntimeWarning: invalid value encountered in double_scalars
  lift = mde_ttl[x][1] / mde_ttl[x][0]-1
100%|██████████| 2/2 [00:55<00:00, 27.98s/it]


In [32]:
experiment_summary = pd.DataFrame(result_dict).T

# Делаем поправку БХ на множественные сравнения

In [33]:
p_res = pd.DataFrame(experiment_summary['p_val']).fillna(1)

In [34]:
p_adj_list = {}

for bonus_id in p_res.index.get_level_values(0).unique():

    loc_index = p_res[p_res.index.get_level_values(0)==bonus_id].index
    loc_p = p_res[p_res.index.get_level_values(0)==bonus_id]['p_val']
    loc_p_adj = multipletests(loc_p, alpha=0.01, method='fdr_bh')[1]
    loc_dict = dict(list(zip(loc_index,zip(loc_p,loc_p_adj))))
    
    p_adj_list.update(loc_dict)

In [35]:
p_adj = pd.DataFrame(p_adj_list, index=['p_val','p_adj']).T

# Итоги теста:

In [36]:
experiment_summary = experiment_summary.join(p_adj['p_adj'])
experiment_summary = experiment_summary.reset_index().rename(columns={"level_0": "bonus_id", "level_1": "split_pair",
                                                                      "level_2": "metric"})
experiment_summary['title'] = experiment_summary['bonus_id'].apply(lambda x: titles_dict.get(x))
experiment_summary = experiment_summary[[
    'bonus_id', 'title', 'split_pair', 'metric', 'split_a', 'split_b', 'lift', 'split_a_n_obs', 'split_b_n_obs',
    'n_obs_difference', 'split_a_drivers', 'split_b_drivers', 'unique_drivers_difference', 'p_val', 'p_adj',
    'nobs_needed', 'mde_ok']]

In [37]:
#словарь для порядка сортировки
m_order = {'AR_Total': 0, 'RR_Total': 1, 'FR_Total': 2, 'AR_ChainOffer': 3, 'RR_ChainOffer': 4, 'FR_ChainOffer': 5,
           'AR_0 - 3 km': 6, 'AR_3 - 6 km': 7, 'AR_6 - 10 km': 8, 'AR_+10 km': 9, 'RR_0 - 3 km': 10,
           'RR_3 - 6 km': 11, 'RR_6 - 10 km': 12, 'RR_+10 km': 13, 'FR_0 - 3 km': 14, 'FR_3 - 6 km': 15,
           'FR_6 - 10 km': 16, 'FR_+10 km': 17, 'OF2R': 18, 'CHURN_7DAY': 19, 'CHURN_14DAY': 20, 'CHURN_21DAY': 21,
           'MPH_GROSS': 22, 'MPH_NET': 23, 'ORGANIC_MPH_GROSS': 24, 'ORGANIC_MPH_NET': 25, 'SHpD': 26, 'TPD': 27,
           'GMVPT': 28, 'COPT': 29, 'DIPT': 30, 'COMPT': 31, 'DIST_MEAN': 32, 'DIST_MEDIAN': 33, 'DE_Total': 34,
           'DE_Park': 35, 'DE_Not_park': 36, 'DE_not_brand': 37, 'DE_brand': 38, 'DE_HIGH': 39, 'DE_TOP': 40}

m_order.update(dict((v,k) for k,v in dict(list(enumerate(sorted(experiment_summary.bonus_id.unique())))).items()))
m_order.update(dict((v,k) for k,v in dict(list(enumerate(sorted(experiment_summary.split_pair.unique())))).items()))

In [38]:
experiment_summary = experiment_summary.sort_values(by=['bonus_id','split_pair','metric'],
                                                    key=lambda x: x.map(m_order))
prokras = experiment_summary[(experiment_summary['p_adj']<=0.01)&(experiment_summary['mde_ok']==1)]

In [39]:
experiment_summary.drop(['unique_drivers_difference'],axis=1)

,bonus_id,title,split_pair,metric,split_a,split_b,lift,split_a_n_obs,split_b_n_obs,n_obs_difference,split_a_drivers,split_b_drivers,p_val,p_adj,nobs_needed,mde_ok
0,50535,Тольятти - 7-Days DxGy distance - c 16.02.2022,A_B,AR_Total,0.750693,0.744488,-0.006205,9739.0,18504.0,0.899990,93.0,114.0,2.604773e-01,5.483734e-01,72317.0,NaN
6,50535,Тольятти - 7-Days DxGy distance - c 16.02.2022,A_B,RR_Total,0.211829,0.222492,0.010664,9739.0,18504.0,0.899990,93.0,114.0,4.084070e-02,1.485116e-01,21846.0,NaN
12,50535,Тольятти - 7-Days DxGy distance - c 16.02.2022,A_B,FR_Total,0.037478,0.033020,-0.004458,9739.0,18504.0,0.899990,93.0,114.0,5.543607e-02,1.705725e-01,27004.0,NaN
1,50535,Тольятти - 7-Days DxGy distance - c 16.02.2022,A_B,AR_ChainOffer,0.761181,0.769898,0.008718,3421.0,6797.0,0.986846,87.0,110.0,3.376114e-01,6.487587e-01,35592.0,NaN
7,50535,Тольятти - 7-Days DxGy distance - c 16.02.2022,A_B,RR_ChainOffer,0.221865,0.219950,-0.001915,3421.0,6797.0,0.986846,87.0,110.0,8.453662e-01,9.430746e-01,700491.0,NaN
13,50535,Тольятти - 7-Days DxGy distance - c 16.02.2022,A_B,FR_ChainOffer,0.016954,0.010152,-0.006803,3421.0,6797.0,0.986846,87.0,110.0,4.590987e-03,3.060658e-02,5359.0,NaN
2,50535,Тольятти - 7-Days DxGy distance - c 16.02.2022,A_B,AR_0 - 3 km,0.927039,0.923361,-0.003678,3262.0,5963.0,0.828020,93.0,112.0,5.494307e-01,8.115858e-01,74400.0,NaN
3,50535,Тольятти - 7-Days DxGy distance - c 16.02.2022,A_B,AR_3 - 6 km,0.874677,0.869603,-0.005074,3096.0,5767.0,0.862726,92.0,114.0,5.176381e-01,8.115858e-01,63350.0,NaN
4,50535,Тольятти - 7-Days DxGy distance - c 16.02.2022,A_B,AR_6 - 10 km,0.603911,0.605952,0.002041,1790.0,3629.0,1.027374,88.0,110.0,9.084003e-01,9.570777e-01,854040.0,NaN
5,50535,Тольятти - 7-Days DxGy distance - c 16.02.2022,A_B,AR_+10 km,0.313011,0.335771,0.022760,1591.0,3145.0,0.976744,90.0,111.0,1.228619e-01,3.191961e-01,6176.0,NaN


In [40]:
prokras.drop(['unique_drivers_difference'],axis=1)

,bonus_id,title,split_pair,metric,split_a,split_b,lift,split_a_n_obs,split_b_n_obs,n_obs_difference,split_a_drivers,split_b_drivers,p_val,p_adj,nobs_needed,mde_ok
25,50535,Тольятти - 7-Days DxGy distance - c 16.02.2022,A_B,SHpD,26.379300,39.021153,0.479234,79.0,93.0,0.177215,79.0,93.0,5.402002e-10,2.160801e-08,59.0,1.0
40,50735,Москва - 7-day DxGy distance - Небренд без К+ ...,A_B,AR_Total,0.516480,0.553583,0.037102,3489.0,12560.0,2.599885,175.0,377.0,1.068581e-04,1.015152e-03,2699.0,1.0
52,50735,Москва - 7-day DxGy distance - Небренд без К+ ...,A_B,FR_Total,0.085411,0.063057,-0.022354,3489.0,12560.0,2.599885,175.0,377.0,4.133600e-06,5.235893e-05,2326.0,1.0
45,50735,Москва - 7-day DxGy distance - Небренд без К+ ...,A_B,AR_+10 km,0.393273,0.464356,0.071083,1546.0,5597.0,2.620310,156.0,326.0,7.607409e-07,1.445408e-05,702.0,1.0
57,50735,Москва - 7-day DxGy distance - Небренд без К+ ...,A_B,FR_+10 km,0.113842,0.074147,-0.039695,1546.0,5597.0,2.620310,156.0,326.0,6.942260e-07,1.445408e-05,952.0,1.0


In [ ]:
#экспорт в эксель
experiment_summary.to_excel(r'res_ttl.xlsx', sheet_name='experiment_summary', index = False)
prokras.to_excel(r'res_prokras.xlsx', sheet_name='prokras', index = False)